In [2]:
import time
import hmac
import hashlib
import requests
import json

# Tuya credentials
API_KEY = 'vtj3ft7hadfea5y3yvs7'
API_SECRET = '16347a3b50984ae696e6887e4d503fa2'
BASE_URL = 'https://openapi.tuyaeu.com'  # This might change depending on your region
TUYA_ENDPOINT = 'https://openapi.tuyaeu.com/v1.0'  

def create_signature(api_key, api_secret, t):
    string_to_sign = f"{api_key}{t}"
    sign = hmac.new(api_secret.encode('utf-8'), msg=string_to_sign.encode('utf-8'), digestmod=hashlib.sha256).hexdigest().upper()
    return sign

# Function to get an access token
def get_access_token(api_key, api_secret):
    t = int(time.time() * 1000)
    sign = create_signature(api_key, api_secret, t)
    headers = {
        'client_id': api_key,
        'sign': sign,
        't': str(t),
        'sign_method': 'HMAC-SHA256'
    }
    response = requests.post(f"{TUYA_ENDPOINT}/token?grant_type=1", headers=headers)
    # response.raise_for_status()
    return response

result = get_access_token(API_KEY, API_SECRET)
print(result)

<Response [200]>


In [10]:
import sys
import hashlib
import hmac
import json
import urllib
import urllib.parse
import logging
import requests
import time

from urllib.request import urlopen, Request
from datetime import datetime

def make_request(url, params=None, headers=None):
    if params:
        url = url + "?" + urllib.parse.urlencode(params)
    request = Request(url, headers=headers or {})

    try:
        with urlopen(request, timeout=10) as response:
            return response, response.read().decode("utf-8")

    except Exception as error:
        return error, ""

def get_timestamp(now = datetime.now()):
    return str(int(datetime.timestamp(now)*1000))

def get_sign(payload, key):
    byte_key = bytes(key, 'UTF-8')
    message = payload.encode()
    sign = hmac.new(byte_key, message, hashlib.sha256).hexdigest()
    return sign.upper()

def generate_signature(client_id, secret, timestamp):
    message = client_id + timestamp
    sign = hmac.new(secret.encode('utf-8'), msg=message.encode('utf-8'), digestmod=hashlib.sha256).hexdigest().upper()
    return sign

def get_access_token():
    now = datetime.now()

    timestamp = get_timestamp(now)
    string_to_sign = client_id + timestamp + "GET\n" + \
        "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855\n" + \
        "\n" + \
        LOGIN_URL
    signed_string = get_sign(string_to_sign, client_secret)

    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    response, body = make_request(BASE_URL + LOGIN_URL, headers = headers)

    json_result = json.loads(body)["result"]
    access_token = json_result["access_token"]
    return access_token

'''
Get the current attributes of device as hash. Example:
{
    "attribute1": value,
    "attribute2": value,
    "attribute3": value
}
'''
def get_device_properties(access_token, device_id):
    url = ATTRIBUTES_URL.format(device_id=device_id)
    timestamp = get_timestamp()
    string_to_sign = client_id + access_token + timestamp + "GET\n" + \
        "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855\n" + \
        "\n" + \
        url
    
    print('aromax: ' + string_to_sign)
    signed_string = get_sign(string_to_sign, client_secret)
    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "access_token": access_token,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    response, body = make_request(BASE_URL + url, headers = headers)

    json_result = json.loads(body)
    properties = json_result["result"]["properties"]
    output = {j['code']: j['value'] for j in properties}

    return output

def set_temperature(access_token, device_id, temperature):
    # Set desired temperature
    headers = {
        "client_id": client_id,
        "access_token": access_token,
        "sign": generate_signature(client_id, client_secret, str(int(time.time() * 1000))),
        "sign_method": "HMAC-SHA256",
        "t": get_timestamp(),
        "mode": "cors"
    }

    # The command must follow Tuya's standard instruction set
    # For example, if the 'code' for setting temperature on your TRV is 'temp_set', use the following payload:
    payload = {
        "commands": [
            {"code": "temp_set", "value": temperature}
        ]
    }

    # Calculate request sign
    sign_str = client_id + access_token + headers["t"]
    headers["sign"] = hmac.new(
        client_secret.encode('utf-8'),
        msg=sign_str.encode('utf-8'),
        digestmod=hashlib.sha256
    ).hexdigest().upper()

    # Send command to device
    command_url = f"{BASE_URL}/v1.0/iot-03/devices/{device_id}/commands"
    response = requests.post(command_url, headers=headers, data=payload)
    response.raise_for_status()
    return response.json()

BASE_URL       = "https://openapi.tuyaeu.com"
LOGIN_URL      = "/v1.0/token?grant_type=1"
ATTRIBUTES_URL = "/v2.0/cloud/thing/{device_id}/shadow/properties"


client_id = "vtj3ft7hadfea5y3yvs7"
client_secret = "16347a3b50984ae696e6887e4d503fa2"
device_id = "bfe22383bb69caf809simm"
access_token  = get_access_token()

result = set_temperature(access_token, device_id, 24)

attributes  = get_device_properties(access_token, device_id)
json_output = json.dumps(attributes)
print(access_token)
print(json_output)

aromax: vtj3ft7hadfea5y3yvs7cb6f0ac868e5167a70d8b07b232eeea61699254091973GET
e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855

/v2.0/cloud/thing/bfe22383bb69caf809simm/shadow/properties
cb6f0ac868e5167a70d8b07b232eeea6
{"mode": "manual", "temp_set": 25, "temp_current": 180, "switch_rapid": false, "countdown_rapid": 0, "work_state": "opened", "window_check": false, "window_state": "closed", "child_lock": false, "battery_percentage": 67, "week_program11": "BgAoCx4qDR4sER4uBgAwDAAuDh4sER4qBgAmDB4oDh4qEh4o", "BoostTime": 300, "Valve_Ratio": 100, "FillTemp": -3, "Eco_Mode": false, "Eco_Temp": 20, "MaxSettemp": 45, "MinSetTEMP": 5}


In [27]:
D = "1699226561104"
E = "CA1E9A479C699AB2B74B6A4F11837A3B58A52E82CCEA8126D3B2D58F14386B0C"

In [ ]:
import requests

# Set up the endpoint and headers based on the cURL command
url = "https://openapi.tuyaeu.com/v2.0/cloud/thing/bfe22383bb69caf809simm/shadow/properties"
headers = {
    "sign_method": "HMAC-SHA256",
    "client_id": "vtj3ft7hadfea5y3yvs7",
    "t": D,
    "mode": "cors",
    "Content-Type": "application/json",
    "sign": E,
    "access_token": "6a7f11dab51faecae86abc551626dc31"
}

# Make the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON if needed
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code, response.text)



In [8]:
import time
import hashlib
import json
import requests


client_id = "vtj3ft7hadfea5y3yvs7"
client_secret = "16347a3b50984ae696e6887e4d503fa2"
device_id = "bfe22383bb69caf809simm"

# Tuya API endpoints
url_get_token = 'https://openapi.tuyaeu.com/v1.0/token?grant_type=1'
url_set_temp = f'https://openapi.tuyaeu.com/v1.0/devices/{device_id}/commands'

# Get the signature required for Tuya API
def get_signature(client_id, client_secret, timestamp):
    string_to_hash = client_id + timestamp
    sign = hashlib.sha256(string_to_hash.encode('utf-8')).hexdigest().upper()
    # For accessing the API, signature needs to be hashed with the Client Secret as well
    secret_hash = hashlib.sha256((client_secret + sign).encode('utf-8')).hexdigest().upper()
    return secret_hash

# Get the current timestamp
timestamp = str(int(time.time() * 1000))

# Generate the sign (signature)
signature = get_signature(client_id, client_secret, timestamp)

# Request headers
headers = {
    'client_id': client_id,
    'sign': signature,
    'sign_method': 'HMAC-SHA256',
    't': timestamp,
    'Content-Type': 'application/json'
}

# Get the access token
response = requests.get(url_get_token, headers=headers)
response_data = response.json()

if response.status_code == 200 and 'result' in response_data:
    access_token = response_data['result']['access_token']
    print("Access Token:", access_token)

    # Command to set the temperature
    commands = {
        'commands': [
            {'code': 'temp_set', 'value': 24}
        ]
    }

    # Update headers with access token
    headers['Authorization'] = f'Bearer {access_token}'

    # Set the temperature
    response = requests.post(url_set_temp, headers=headers, json=commands)
    if response.status_code == 200:
        print("Temperature set successfully.")
    else:
        print("Failed to set temperature.", response.text)
else:
    print("Failed to get access token.", response_data)


Failed to get access token. {'code': 1004, 'msg': 'sign invalid', 'success': False, 't': 1699254004851, 'tid': '1ce4747d7c7211eeab9046acc027565a'}
